In [12]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-08-19 20:13:24.382581


In [13]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collections

In [3]:
%load_ext autoreload
%autoreload
col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Artists

In [17]:
%load_ext autoreload
%autoreload

def parseArtists(arts, force=False):
    for modVal in range(18,19):
        arts.parseArtistModValFiles(modVal, force=force)
        arts.parseArtistModValExtraFiles(modVal)

from artist import artist
from artists import artists
arts = artists(disc)

parseArtists(arts, True)
#arts.parseArtistFiles(force=False)
#if False:
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata(force=False, doAlbums=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Parsing Artist Files For ModVal 18
Forcing Reloads of ModVal=18
  Processing 4584 files.
	Processed 0/4584 files.
	Processed 250/4584 files.
	Processed 500/4584 files.
	Processed 750/4584 files.
	Processed 1000/4584 files.
	Processed 1250/4584 files.
	Processed 1500/4584 files.
	Processed 1750/4584 files.
	Processed 2000/4584 files.
	Processed 2250/4584 files.
	Processed 2500/4584 files.
	Processed 2750/4584 files.
	Processed 3000/4584 files.
	Processed 3250/4584 files.
	Processed 3500/4584 files.
	Processed 3750/4584 files.
	Processed 4000/4584 files.
	Processed 4250/4584 files.
	Processed 4500/4584 files.
Saving 4584 new artist IDs to /Volumes/Music/Discog/artists-db/18-DB.p
  --> This file is 8.9MB.
Parsing Artist Extra Files For ModVal 18
  Found 6 extra files for ModVal 18
  Loaded 4584 artist IDs.
27518 	 500	1000
27518 	 1000	1500
27518 	 1500	2000
27518 	 2000	2500
27518 	 2500	3000
27518 	 

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)
if True:
    albs.parseAlbumModValFiles(modVal='NAN', force=False)
    for modVal in range(100):
        albs.parseAlbumModValFiles(modVal=modVal, force=False)
albs.buildMetadata(force=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Tue Aug 20, 2019 19:36:55 for Parsing Album Files For ModVal NAN
  Looking for artist albums in /Volumes/Music/Discog/albums/NAN
  Found 1 artist directories
    Found 39809 albums (0    ) in 0/1	/Volumes/Music/Discog/albums/NAN/NAN
Saving 3790 new album IDs to /Volumes/Music/Discog/albums-db/NAN-DB.p
  --> This file is 37.9MB.
Current Time is Tue Aug 20, 2019 19:46:40 for Done with Parsing Album Files For ModVal NAN
Process [{0}] took 9.7 minutes.
Current Time is Tue Aug 20, 2019 19:46:41 for Parsing Album Files For ModVal 0
  Looking for artist albums in /Volumes/Music/Discog/albums/0
  Found 2390 artist directories
    Found 6   albums (0    ) in 0/2390	/Volumes/Music/Discog/albums/0/1000
    Found 2   albums (0    ) in 100/2390	/Volumes/Music/Discog/albums/0/1155300
    Found 3   albums (0    ) in 200/2390	/Volumes/Music/Discog/albums/0/1322100
    Found 0   albums (0    ) in 300

    Found 1   albums (8    ) in 100/2606	/Volumes/Music/Discog/albums/3/1152003
    Found 0   albums (18   ) in 200/2606	/Volumes/Music/Discog/albums/3/132003
    Found 2   albums (32   ) in 300/2606	/Volumes/Music/Discog/albums/3/146003
    Found 1   albums (33   ) in 400/2606	/Volumes/Music/Discog/albums/3/1602303
    Found 1   albums (78   ) in 500/2606	/Volumes/Music/Discog/albums/3/1759403
    Found 5   albums (85   ) in 600/2606	/Volumes/Music/Discog/albums/3/1944003
    Found 3   albums (129  ) in 700/2606	/Volumes/Music/Discog/albums/3/209603
    Found 1   albums (136  ) in 800/2606	/Volumes/Music/Discog/albums/3/2285703
    Found 11  albums (143  ) in 900/2606	/Volumes/Music/Discog/albums/3/248403
    Found 1   albums (168  ) in 1000/2606	/Volumes/Music/Discog/albums/3/2652803
    Found 1   albums (181  ) in 1100/2606	/Volumes/Music/Discog/albums/3/285603
    Found 8   albums (188  ) in 1200/2606	/Volumes/Music/Discog/albums/3/3067403
    Found 18  albums (207  ) in 1300/2606	

    Found 0   albums (228  ) in 1000/1264	/Volumes/Music/Discog/albums/6/5424706
    Found 0   albums (251  ) in 1100/1264	/Volumes/Music/Discog/albums/6/645106
    Found 5   albums (261  ) in 1200/1264	/Volumes/Music/Discog/albums/6/828006
    Found 1   albums (281  ) in 1263/1264	/Volumes/Music/Discog/albums/6/997006
Saving 281 new album IDs to /Volumes/Music/Discog/albums-db/6-DB.p
  --> This file is 2.9MB.
Current Time is Tue Aug 20, 2019 19:51:29 for Done with Parsing Album Files For ModVal 6
Process [{0}] took 43 seconds.
Current Time is Tue Aug 20, 2019 19:51:29 for Parsing Album Files For ModVal 7
  Looking for artist albums in /Volumes/Music/Discog/albums/7
  Found 1319 artist directories
    Found 7   albums (0    ) in 0/1319	/Volumes/Music/Discog/albums/7/100507
    Found 0   albums (28   ) in 100/1319	/Volumes/Music/Discog/albums/7/1342007
    Found 1   albums (43   ) in 200/1319	/Volumes/Music/Discog/albums/7/1664407
    Found 1   albums (66   ) in 300/1319	/Volumes/Music/

  Found 1342 artist directories
    Found 3   albums (0    ) in 0/1342	/Volumes/Music/Discog/albums/12/1002012
    Found 0   albums (36   ) in 100/1342	/Volumes/Music/Discog/albums/12/124012
    Found 0   albums (48   ) in 200/1342	/Volumes/Music/Discog/albums/12/1509612
    Found 7   albums (75   ) in 300/1342	/Volumes/Music/Discog/albums/12/189012
    Found 1   albums (87   ) in 400/1342	/Volumes/Music/Discog/albums/12/227712
    Found 21  albums (124  ) in 500/1342	/Volumes/Music/Discog/albums/12/264812
    Found 1   albums (143  ) in 600/1342	/Volumes/Music/Discog/albums/12/3078612
    Found 0   albums (148  ) in 700/1342	/Volumes/Music/Discog/albums/12/3465512
    Found 0   albums (160  ) in 800/1342	/Volumes/Music/Discog/albums/12/3936012
    Found 1   albums (174  ) in 900/1342	/Volumes/Music/Discog/albums/12/4554612
    Found 1   albums (190  ) in 1000/1342	/Volumes/Music/Discog/albums/12/5101012
    Found 3   albums (208  ) in 1100/1342	/Volumes/Music/Discog/albums/12/5817412


In [15]:
ifile='/Volumes/Music/Discog/albums/NAN/NAN/10000892.p'
retval = albs.getData(ifile)

In [22]:
retval.album.name

'Africa In America (Rock, Jazz & Calypso 1920 -1962)'

In [16]:
for k,v in db.items():
    print(k,v)
    break

Dave Clarke 1000


In [ ]:
f1="/Volumes/Music/Discog/albums/6/102506/482739.p"
#f2="/Volumes/Music/Discog/albums/6/102506/7303693.p"
#albData1 = alb.getData(f1)
f2="/Volumes/Music/Discog/albums/11/567511/3386791.p"
albData2 = alb.getData(f2)

In [ ]:
albData1.album

In [ ]:
for k,v in albData2.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
for k,v in albData1.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
from album import album
from fileUtils import getsize
alb = album()
for ifile in files:
    print(getsize(ifile))
    #alb.getData(ifile)

In [ ]:
for modVal in range(58,59):
    arts.parseArtistModValFiles(modVal=modVal, force=False)

dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
from time import sleep
refs = [dbCol.get('716158')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if True:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_mp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map(f, chunks)
 
    return sum(result)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
 
    while not result.ready():
        print("Running...")
        time.sleep(0.5)
 
    return sum(result.get())

In [ ]:
L = [{0: True}, {1: True}, {2: True}, {3: True}, {4: True}]

In [ ]:
{k: v for d in L for k, v in d.items()}

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()
artistInfo = artist()
artistDBDir = disc.getArtistsDBDir()   

In [ ]:
len(files)

In [ ]:
from fileUtils import getBaseFilename
import time
from multiprocessing import Pool

def f(finput):
    a_list = finput["Data"]
    dbdata = finput["DB"]
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out

def fDB(modVal):
    saveIt = arts.parseArtistModValFiles(modVal, force=True)
    
    if False:
        from searchUtils import findExt
        files = findExt(setDir(artistDir, str(modVal)), ext="*.p")
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        dbdata = getFile(dbname, version=3)

        out = []
        saveIt = 0
        for ifile in files:
            discID = getBaseFilename(ifile)
            if dbdata.get(discID) is None:
                info   = artistInfo.getData(ifile)
                dbdata[discID] = info
                saveIt += 1

        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
        
    return saveIt

 
def f_ampFiles(a_list, dbdata):
    nProcs = 1
    chunks = [a_list[i::nProcs] for i in range(nProcs)]

    for c in chunks:
        print(len(c))
    
    pool = Pool(processes=nProcs)
    
    
 
    finputs = [{"Data": chunk, "DB": dbdata} for chunk in chunks]
    result = pool.map_async(f, finputs)
 
    start, cmt = clock("Processing {0} files using {1} processes".format(len(files), nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()



 
def f_amp(modVals):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(fDB, range(3,modVals))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
from timeUtils import clock, elapsed
retval = f_amp(100)
#flat_list = [item for sublist in retval for item in sublist]
#print(len(flat_list))

In [ ]:
dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
dbdata = getFile(dbname, version=3)

In [ ]:
modVals = 1
nProcs = 1
start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
print("Runing .",end="")
retval = fDB(modVals)
print("")
elapsed(start, cmt)


In [ ]:
retval

In [ ]:
vals="""1,708,466 US
1,708,466 UK
1,154,284 Germany
637,261 France
493,079 Japan
490,698 Italy
434,681 Canada
433,543 Europe
416,965 Netherlands
339,738 Spain
320,714 Unknown
259,587 Russia
259,579 Australia
189,431 Sweden
164,278 Belgium
149,548 Brazil
131,106 Greece
124,488 Poland
121,249 Jamaica
105,887 Finland
86,538 Switzerland
85,145 USSR
80,203 Mexico
79,972 Denmark
77,131 Portugal
75,979 Norway
75,533 UK & Europe
67,292 Austria
63,672 Argentina"""

In [ ]:
countries = [" ".join(x.split()[1:]) for x in vals.split("\n")]
saveFile(idata=countries, ifile="countries.p")

In [ ]:
countries